___


___
# Text Generation



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
# IGNORE THE CONTENT OF THIS CELL
# tf.compat.v1.disable_eager_execution()

In [4]:
tf.__version__

'2.0.0'

## Step 1: The Data



In [5]:
path_to_file = 'shakespeare.txt'

In [6]:
text = open(path_to_file, 'r').read()

In [7]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Understanding unique characters

In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab) #this length will be important for the last dense layer

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

## Step 2: Text Processing

In [9]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [10]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [11]:
ind_to_char = np.array(vocab)

In [12]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [13]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [14]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

We now have a mapping we can use to go back and forth from characters to numerics.

In [15]:
sample = text[:20]
sample

'\n                   '

In [16]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Step 3: Creating Batches



In [17]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [18]:
line = "From fairest creatures we desire increase"

In [19]:
len(line)

41

In [20]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""
#hence we are going to take 3 lines as it is enough to reflect the rhyming scheme but this varies depending on the type pof text provided

In [21]:
len(part_stanza)
#hence we take our sequence length to be 120 and train our model on these sequences of length 131

131

### Training Sequences



In [22]:
seq_len = 120

In [23]:
total_num_seq = len(text)//(seq_len+1)

In [24]:
total_num_seq

45005

In [30]:
# Create Training Sequences
#the dataset is created from the encoded text
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


In [31]:
#The batch method converts these individual character calls into sequences we can 
#feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:
#in case the sequence length does not completey divide up the total text drop the remainder
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [32]:
def create_seq_targets(seq):
    input_txt = seq[:-1] #hello my nam
    target_txt = seq[1:] #ello my name
    return input_txt, target_txt

In [33]:
dataset = sequences.map(create_seq_targets)

In [34]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches



In [35]:
# Batch size - no of sequences the model will be trained in at a time
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [36]:
dataset #basically a 128 sequences where each sequence is 120 char long

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Step 4: Creating the Model

In [37]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Setting up Loss Function



In [39]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [40]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)



https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [41]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [42]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [43]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


## Step 5: Training the model





In [45]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [46]:
example_batch_predictions

<tf.Tensor: id=5805, shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 3.87407467e-03, -4.80890216e-04,  6.00470416e-03, ...,
          1.89803634e-03,  2.75372760e-03, -4.26887127e-04],
        [-2.20246310e-03, -7.00643240e-03,  1.49408553e-03, ...,
         -3.15369200e-03,  4.09174897e-03,  1.16246077e-03],
        [-7.64427334e-03, -3.40333791e-03,  7.36198330e-04, ...,
         -1.07077870e-03, -1.26574119e-03,  1.30257022e-03],
        ...,
        [-6.68676058e-03, -9.93404258e-03,  6.06101472e-03, ...,
         -8.31529684e-03, -2.28054728e-03, -6.78048190e-03],
        [-1.22171175e-03, -3.78384907e-03,  8.45137797e-03, ...,
          7.21718464e-03,  3.66549846e-03,  1.14446285e-03],
        [ 3.39653809e-03, -4.67280857e-03, -8.80432432e-04, ...,
          9.94975399e-03,  5.44371724e-05,  4.71927924e-03]],

       [[ 2.20951228e-03, -2.68700370e-03, -2.91137490e-03, ...,
          9.93939815e-04, -3.70976422e-03,  2.18067388e-03],
        [ 2.35284935e-03, -6.05265796e

In [47]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [48]:
sampled_indices

<tf.Tensor: id=5811, shape=(120, 1), dtype=int64, numpy=
array([[ 9],
       [11],
       [34],
       [53],
       [22],
       [75],
       [26],
       [81],
       [18],
       [28],
       [31],
       [51],
       [54],
       [28],
       [29],
       [35],
       [31],
       [75],
       [57],
       [31],
       [82],
       [16],
       [69],
       [64],
       [ 4],
       [12],
       [23],
       [21],
       [30],
       [60],
       [58],
       [56],
       [26],
       [78],
       [57],
       [22],
       [76],
       [49],
       [71],
       [ 6],
       [43],
       [14],
       [ 3],
       [30],
       [ 2],
       [35],
       [53],
       [ 8],
       [ 5],
       [65],
       [ 3],
       [74],
       [20],
       [ 1],
       [44],
       [56],
       [ 8],
       [56],
       [78],
       [82],
       [59],
       [44],
       [24],
       [67],
       [40],
       [33],
       [54],
       [57],
       [70],
       [48],
       [ 4],
       [51],
       

In [49]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [50]:
sampled_indices

array([ 9, 11, 34, 53, 22, 75, 26, 81, 18, 28, 31, 51, 54, 28, 29, 35, 31,
       75, 57, 31, 82, 16, 69, 64,  4, 12, 23, 21, 30, 60, 58, 56, 26, 78,
       57, 22, 76, 49, 71,  6, 43, 14,  3, 30,  2, 35, 53,  8,  5, 65,  3,
       74, 20,  1, 44, 56,  8, 56, 78, 82, 59, 44, 24, 67, 40, 33, 54, 57,
       70, 48,  4, 51, 11, 34, 74, 61, 73,  2, 14, 22, 77, 80, 81, 51, 27,
       32, 79, 35, 80, 61, 24,  3, 12, 64, 62, 75, 11, 54, 46, 42, 81, 66,
       51, 66,  2, 80, 11, 13, 26, 79, 44, 55, 81, 67, 35, 74,  0, 57, 57,
       73])

In [51]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 

sport with the fox ere we case him.
    He was first smok'd by the old Lord Lafeu. When his disguise and
    he is parte


Next Char Predictions: 

-0I];tAz7CFZ_CDJFtbF|5ni&1<:EecaAwb;uXp(R3"E!J],'j"s9 Sa,aw|dS>lOH_boW&Z0Isfr!3;vyzZBGxJyf>"1igt0_UQzkZk!y02AxS`zlJs
bbr


After confirming the dimensions are working, let's train our network!

In [52]:
epochs = 30

In [53]:
model.fit(dataset,epochs=epochs)

Epoch 1/30
351/351 [==============================] - 1358s 4s/step - loss: 2.5899
Epoch 2/30
106/351 [========>.....................] - ETA: 16:41 - loss: 1.9619

KeyboardInterrupt: 

## Step 6: Generating text



In [0]:
model.save('shakespeare_gen.h5') 

In [0]:
from tensorflow.keras.models import load_model

In [0]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))


In [0]:
model.summary()

In [0]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [0]:
print(generate_text(model,"flower",gen_size=1000))